## Numeryczne Rozwiązywanie Rónań Różniczkowych Cząstkowych
### Uniwersytet Jagielloński 2021
Katedra Teorii Optymalizacji i Sterowania UJ



# Metoda różnic skończonych


## Laboratorium V
### Warunek brzegowy Neumanna

$$ u_{xx}  + u_{yy} = 0$$

$$ u_{| \partial{D}} = 0 \,\,\,\,\, \text{for } x = 1 \text{ and } 0 \leq y <1$$
$$ u_{| \partial{D}} = 1 \,\,\,\,\, \text{for } y = 1 \text{ and } 0 \leq x <1$$

$$ \frac{\partial u}{\partial x} = 0 \,\,\,\,\, \text{for } x = 0 \text{ and } 0 \leq y <1$$
$$ \frac{\partial u}{\partial y} = 0 \,\,\,\,\, \text{for } y = 0 \text{ and } 0 \leq x <1$$

$$ D = \{(x, y) \in R^2 : x, y \in (0, 1) \} $$

Rozwiązanie dokładne: brak :(

In [ ]:
import numpy as np
import scipy as sc
import scipy.sparse.linalg
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from scipy.sparse import lil_matrix, csr_matrix
from typing import Optional
import time

In [ ]:
import scipy.sparse.linalg

class SetupElliptic:
    def __init__(self, dx: float, dy: float, f, x_range, y_range, boundary_right, boundary_up, boundary_left, boundary_down):
        self.x_range = x_range
        self.y_range = y_range
        self.boundary_left = boundary_left
        self.boundary_right = boundary_right
        self.boundary_up = boundary_up
        self.boundary_down = boundary_down        

        self.f = f
        self.x_num = round((self.x_range[1] - self.x_range[0]) / dx) + 1
        self.y_num = round((self.y_range[1] - self.y_range[0]) / dy) + 1
        self.X, self.dx = np.linspace(*self.x_range, self.x_num, retstep=True)
        self.Y, self.dy = np.linspace(*self.y_range, self.y_num, retstep=True)

def scheme_elliptic(setup):
    ex = np.ones(setup.x_num-1)
    A = scipy.sparse.dia_matrix((np.array([-1/setup.dx**2 * ex, 2*(1/setup.dx**2 + 1/setup.dy**2) * ex, -1/setup.dx**2 * ex]), [-1, 0, 1]), shape=(setup.x_num-1, setup.x_num-1))
    A[0,0] = -1/setup.dx
    A[0,1] = 1/setup.dx
    B = scipy.sparse.dia_matrix((-1/setup.dy**2 * ex, [0]), shape=(setup.x_num-2, setup.x_num-2))
    X_data = [ [None]*(setup.y_num-2) for i in range(setup.y_num-2)]
    X_data[0][0] = A
    for i in range(1,setup.y_num-2):
        X_data[i][i] = A
        X_data[i-1][i] = B
        X_data[i][i-1] = B
    X = scipy.sparse.bmat(X_data, format="csc")
    f_vec = np.zeros((setup.x_num-2) * (setup.y_num-2))
    n = 0
    for j in range(setup.y_num-2):
        for i in range(setup.x_num-2):
            f_vec[n] += setup.f(setup.X[i+1], setup.Y[j+1])
            if (i == 0):
                f_vec[n] += 1/setup.dx**2 * setup.boundary_left(setup.Y[j+1])
            if (i == setup.x_num-3):
                f_vec[n] += 1/setup.dx**2 * setup.boundary_right(setup.Y[j+1])
            if (j == 0):
                f_vec[n] += 1/setup.dy**2 * setup.boundary_down(setup.X[i+1])
            if (j == setup.y_num-3):
                f_vec[n] += 1/setup.dy**2 * setup.boundary_up(setup.X[i+1])
            n+=1

    u = scipy.sparse.linalg.spsolve(X, f_vec).reshape(setup.y_num-2, setup.x_num-2) # od 1 do Nx-1, 1 do Ny-1
    r = np.zeros((setup.y_num, setup.x_num))
    for j in range(setup.y_num):
        r[j,0] = setup.boundary_left(setup.Y[j])
        r[j,setup.x_num-1] = setup.boundary_right(setup.Y[j])
    for i in range(setup.x_num):
        r[0,i] = setup.boundary_down(setup.X[i])
        r[setup.y_num-1,i] = setup.boundary_up(setup.X[i])
    r[1:setup.y_num-1, 1:setup.x_num-1] = u
    return setup.X, setup.Y, r

In [ ]:
def scheme_elliptic(setup):
    size = (setup.x_num + 1) * (setup.y_num + 1)
    A = lil_matrix((size, size))
    hx = 1 / setup.dx ** 2
    hy = 1 / setup.dy ** 2
    r = -2 * (hx + hy)

    d = np.zeros(size)
    s = setup.y_num + 1
    for i in range(0, setup.x_num + 1):
        for j in range(0, setup.y_num + 1):
            # TODO

    A = csr_matrix(A)
    w = sc.sparse.linalg.spsolve(A, d)
    w = w.reshape(setup.x_num + 1, setup.y_num + 1)

    return w

In [ ]:
def plot_surface(values: float, setup):
    fig = plt.figure()
    ax = fig.gca(projection='3d')

    # Prepare grid.
    X = np.linspace(*setup.x_range, setup.x_num + 1)
    Y = np.linspace(*setup.y_range, setup.y_num + 1)
    X, Y = np.meshgrid(X, Y)

    # Plot the surface.
    surf = ax.plot_surface(X, Y, values, cmap=cm.coolwarm,
                           linewidth=0, antialiased=False)

    # Customize the z axis.
    # ax.set_zlim(-1.01, 1.01)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    # Add a color bar which maps values to colors.
    fig.colorbar(surf, shrink=0.5, aspect=5)

    plt.show()

In [ ]:
%%time
# numerical solution
setup_elliptic = SetupElliptic(0.005)
numerical_elliptic = scheme_elliptic(setup_elliptic)

In [ ]:
plot_surface(numerical_elliptic, setup_elliptic)

# Rozważyć warunki brzegowe (Dirichleta i Neumanna) różne od zera

